In [17]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [36]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()#继承父类初始化
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)
    
    def forward(self, x):#x 1*28*28
        x = F.relu(self.conv1(x)) # 20*24*24
        x = F.max_pool2d(x, 2, 2) # 20*12*12
        x = F.relu(self.conv2(x)) # 50*8*8
        x = F.max_pool2d(x, 2, 2) # 50*4*4
        x = x.view(-1,4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dmi=1)
        return F.log_softmax(x, dim=1) # log probability

In [37]:
mnist_data = datasets.MNIST("./mnist_data", train=True, download=True,
                           transform=transforms.Compose([
                               transforms.ToTensor(),
                           ]))
mnist_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./mnist_data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
           )

In [38]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    #enumerate 用于将一个可遍历的数据对象作为一个索引序列
    for idx, (data,target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        pred = model(data) #benchsize * 10
        loss = F.nll_loss(pred, target)#分类的lossfunction
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if idx % 100 == 0:
           print("Train Epoch: {}, iteration: {}, Loss: {}".format(
           epoch, idx, loss.item()))

In [39]:
def test(model, device, test_loader):
    model.eval()#测试模式
    total_loss = 0.
    correct = 0.
    with torch.no_grad():
        for idx, (data,target) in enumerate(test_loader):
            data, target = data.to(device), target.to(device)
            output = model(data) #benchsize * 10
            #测试集计算总的损失
            total_loss += F.nll_loss(output, target, reduction="sum").item()#分类的lossfunction
            pred = output.argmax(dim=1)
            correct += pred.eq(target.view_as(pred)).sum().item()
        
    
    total_loss /= len(test_loader.dataset)
    acc = correct/len(test_loader.dataset) * 100.
    print("Test loss: {}, Accuracy: {}".format(total_loss, acc))

In [40]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 32
train_dataloader = torch.utils.data.DataLoader(
      datasets.MNIST("./mnist_data", train=True, download=True,
                           transform=transforms.Compose([
                               transforms.ToTensor(),
                               #Normalize 减去均值和方差
                               transforms.Normalize((0.1307,), (0.3081,))
                           ])),
      batch_size=batch_size, shuffle=True,
      num_workers=1, pin_memory=True
)
test_dataloader = torch.utils.data.DataLoader(
      datasets.MNIST("./mnist_data", train=False, download=True,
                           transform=transforms.Compose([
                               transforms.ToTensor(),
                               #Normalize 减去均值和方差
                               transforms.Normalize((0.1307,), (0.3081,))
                           ])),
      batch_size=batch_size, shuffle=True,
      num_workers=1, pin_memory=True
)
lr = 0.01
momentum = 0.5
model = Net().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum)
num_epochs = 2
for epoch in range(num_epochs):
    train(model, device, train_dataloader, optimizer, epoch)
    test(model, device, test_dataloader)
torch.save(model.state_dict(), "mnist_cnn.pt")

Train Epoch: 0, iteration: 0, Loss: 2.2941701412200928
Train Epoch: 0, iteration: 100, Loss: 0.4841163158416748
Train Epoch: 0, iteration: 200, Loss: 0.28404948115348816
Train Epoch: 0, iteration: 300, Loss: 0.41409358382225037
Train Epoch: 0, iteration: 400, Loss: 0.13008913397789001
Train Epoch: 0, iteration: 500, Loss: 0.27093514800071716
Train Epoch: 0, iteration: 600, Loss: 0.2826293706893921
Train Epoch: 0, iteration: 700, Loss: 0.1390560120344162
Train Epoch: 0, iteration: 800, Loss: 0.1593220978975296
Train Epoch: 0, iteration: 900, Loss: 0.1107160747051239
Train Epoch: 0, iteration: 1000, Loss: 0.08384168148040771
Train Epoch: 0, iteration: 1100, Loss: 0.053495556116104126
Train Epoch: 0, iteration: 1200, Loss: 0.07284262776374817
Train Epoch: 0, iteration: 1300, Loss: 0.012579858303070068
Train Epoch: 0, iteration: 1400, Loss: 0.02969057857990265
Train Epoch: 0, iteration: 1500, Loss: 0.10437951982021332
Train Epoch: 0, iteration: 1600, Loss: 0.019353672862052917
Train Epoch: